In [22]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time 
import pandas as pd
import pymongo
# Initialize the webdriver
# de manière basique
# driver = webdriver.Chrome()

# avec ChromeDriverManager mieux car update du driver automatiquement
driver = webdriver.Chrome(ChromeDriverManager().install())

# si on a télécharger le driver directement dans le dossier 
# service = Service(executable_path="./chromedriver.exe")
# driver = webdriver.Chrome(service=service)

C:\Users\camil\AppData\Local\Temp\ipykernel_13440\86999936.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [23]:
# Navigate to the website
# driver.get("https://www.billboard.com/charts/hot-100/")
driver.get("https://www.billboard.com/charts/billboard-global-200/")

# Wait for the page to load
driver.implicitly_wait(10)

# reject all cookies
driver.find_element(By.XPATH,"/html/body/div[6]/div[2]/div/div/div[2]/div/div/button[1]").click()

In [24]:
iterateur = list(range(1,110,11))
iterateur.remove(1)
iterateur200 = list(range(1,220,11))
iterateur200.remove(1)

In [25]:
def scraper(list,xpath,iterateur,nb_max):
    for i in range(2,nb_max):
        if i not in iterateur : 
            WebElement = driver.find_element(By.XPATH,xpath%(i))
            list.append(WebElement.text)

    return list

In [26]:
XpathTitle = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[1]/h3"
XpathArtist = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[1]/span"
XpathRank = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[1]/span "
XpathLastWeek = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[4]/span"
XpathPeakPosition = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[5]/span"
XpathWeeksOnChart = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[6]/span"

In [27]:
allXpath = [XpathTitle,XpathArtist,XpathRank,XpathLastWeek,XpathPeakPosition,XpathWeeksOnChart]

In [28]:
title,artist,rank,last_week,peak_pos,weeks_on_chart = [],[],[],[],[],[]
allList = [title,artist,rank,last_week,peak_pos,weeks_on_chart]

In [29]:
# for i, j in zip(allList, allXpath):
#     i = scraper(i,j,iterateur200,221)

In [30]:
i = [scraper(i,j,iterateur200,221) for i, j in zip(allList, allXpath)]

In [31]:
df = pd.DataFrame(list(zip(title,artist,rank,last_week,peak_pos,weeks_on_chart)),columns=['Title','Artist','Rank','Last Week','Peak Positon','Weeks on charts'])
df

,Title,Artist,Rank,Last Week,Peak Positon,Weeks on charts
0,Flowers,Miley Cyrus,1,1,1,2
1,"Bzrp Music Sessions, Vol. 53",Bizarrap & Shakira,2,2,2,3
2,Kill Bill,SZA,3,3,1,7
3,Calm Down,Rema & Selena Gomez,4,4,3,30
4,Unholy,Sam Smith & Kim Petras,5,6,1,18
...,...,...,...,...,...,...
195,Wasted On You,Morgan Wallen,196,-,19,49
196,Gatita,Bellakath,197,147,113,6
197,Wake Me Up!,Avicii,198,200,154,34
198,Only Love Can Hurt Like This,Paloma Faith,199,-,75,26


In [32]:
# Close the webdriver
driver.close()

In [33]:
result = df.to_dict('records')

In [34]:
type(result)

list

In [35]:
result

[{'Title': 'Flowers',
  'Artist': 'Miley Cyrus',
  'Rank': '1',
  'Last Week': '1',
  'Peak Positon': '1',
  'Weeks on charts': '2'},
 {'Title': 'Bzrp Music Sessions, Vol. 53',
  'Artist': 'Bizarrap & Shakira',
  'Rank': '2',
  'Last Week': '2',
  'Peak Positon': '2',
  'Weeks on charts': '3'},
 {'Title': 'Kill Bill',
  'Artist': 'SZA',
  'Rank': '3',
  'Last Week': '3',
  'Peak Positon': '1',
  'Weeks on charts': '7'},
 {'Title': 'Calm Down',
  'Artist': 'Rema & Selena Gomez',
  'Rank': '4',
  'Last Week': '4',
  'Peak Positon': '3',
  'Weeks on charts': '30'},
 {'Title': 'Unholy',
  'Artist': 'Sam Smith & Kim Petras',
  'Rank': '5',
  'Last Week': '6',
  'Peak Positon': '1',
  'Weeks on charts': '18'},
 {'Title': "Creepin'",
  'Artist': 'Metro Boomin, The Weeknd & 21 Savage',
  'Rank': '6',
  'Last Week': '5',
  'Peak Positon': '3',
  'Weeks on charts': '8'},
 {'Title': 'Escapism',
  'Artist': 'RAYE Featuring 070 Shake',
  'Rank': '7',
  'Last Week': '7',
  'Peak Positon': '7',
  'We

In [36]:
from pymongo import MongoClient

# Connect to the MongoDB database
client = MongoClient("mongodb://localhost:27017/")
db = client["mydatabase"]
collection = db["mycollection"]

In [37]:
print(type(client))
print(type(db))
print(collection)

<class 'pymongo.mongo_client.MongoClient'>
<class 'pymongo.database.Database'>
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'mydatabase'), 'mycollection')


In [38]:
collection.insert_many(result)

In [39]:
collection.find_one()

{'_id': ObjectId('63dfc2564ea09775f274fa84'),
 'Title': 'Flowers',
 'Artist': 'Miley Cyrus',
 'Rank': '1',
 'Last Week': '1',
 'Peak Positon': '1',
 'Weeks on charts': '2'}